In [14]:
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
import pandas as pd
import re

In [18]:
def substring_range(s, substring):
    for i in re.finditer(re.escape(substring), s):
        yield (i.start(), i.end())

In [4]:
df = pd.read_csv('/Users/tozturk/Desktop/HAIRCAIR/Codes/RedditScraper/TextData.csv')
productList = pd.read_csv('/Users/tozturk/Desktop/HairProducts')

In [46]:
i1 = str(df.body[1])
i2 = str(df.body[2])
i3 = str(df.body[4])
i4 = str(df.body[5])
i4

"I’m still basically on the beginners routine but I’ll include my little changes!\n\n1. Shampooed with dippity doo curl shampoo because I have fine hair and I find co-washing works for me about half the time and I need to shampoo the other half of the time\n2. Applied rice water to my hair and wet plopped for ten-ish minutes and rinsed\n3. Applied a handful of the tresseme botanical conditioner and gently finger combed my hair to get out knots. Sometimes I comb and repart my hair but I didn’t need to today\n4. Flipped my head over and squished to condish, but then I have to flip it upright to comb a little at the crown of my head, because otherwise I get big patchy spots lol\n5. Prayer hands applied herbal essences totally twisted gel and then scrunched\n6. Wrapped my hair up in a microfiber towel turban so I can use body wash and just generally get all the hair product off my skin before getting out of the shower\n7. Scrunched out as much water as possible with the towel turban\n8. Di

In [44]:
print([x for x in substring_range(i3, "rosemary oil")])

[(814, 826)]


In [53]:
i = df.body[1]

# training data
TRAIN_DATA = [
    (i1, {"entities": [(35, 72, "ORG"), (75, 101, "ORG")]}),
    (i2, {"entities": [(113, 142, "ORG"), (308, 337, "ORG"), (152, 200, "ORG"), (202, 245, "ORG"),
                       (308,357, "ORG"), (483, 510, "ORG"), (519, 541, "ORG"), (729, 769, "ORG")]}),
    (i3, {"entities": [(353, 377, "ORG"), (760, 776, "ORG"), (799, 809, "ORG"),
                      (814, 826, "ORG")]}), 
]


@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])


if __name__ == "__main__":
    plac.call(main(model =  'en'))
    
    

Loaded model 'en'
Losses {'ner': 6.025852323638901}
Losses {'ner': 5.169803202639741}
Losses {'ner': 3.376199821204864}
Losses {'ner': 2.3826443790521807}
Losses {'ner': 3.278167609200338}
Losses {'ner': 2.531080874730833}
Losses {'ner': 2.793932954916386}
Losses {'ner': 1.8526069099598566}
Losses {'ner': 2.543793841472052}
Losses {'ner': 2.510759112245111}
Losses {'ner': 2.7870500242705702}
Losses {'ner': 2.65725796598781}
Losses {'ner': 2.7056617249327246}
Losses {'ner': 3.3047609198869736}
Losses {'ner': 2.185876369791231}
Losses {'ner': 1.738854438398107}
Losses {'ner': 2.208912010179745}
Losses {'ner': 3.001346165624909}
Losses {'ner': 1.95690814628324}
Losses {'ner': 1.6649432080922593}
Losses {'ner': 1.1770130825596254}
Losses {'ner': 2.309761112993444}
Losses {'ner': 1.3872991871807336}
Losses {'ner': 1.5121536562023548}
Losses {'ner': 1.1455488057659144}
Losses {'ner': 1.6723977007389266}
Losses {'ner': 1.6369934211408506}
Losses {'ner': 1.7616610759139049}
Losses {'ner': 1.37

TypeError: Could not determine the signature of None

In [1]:
import nltk

In [ ]:
sentence = "the little yellow dog barked at the cat"
grammar = ('''
    NP: {<DT>?<JJ>*<NN>} # NP
    ''')

chunkParser = nltk.RegexpParser(grammar)
tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
tagged